In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE93101"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE93101"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE93101.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE93101.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE93101.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular Prognosis of Cardiogenic Shock Patients under Extracorporeal Membrane Oxygenation"
!Series_summary	"Prognosis for cardiogenic shock patients under ECMO was our study goal. Success defined as survived more than 7 days after ECMO installation and failure died or had multiple organ failure in 7 days. Total 34 cases were enrolled, 17 success and 17 failure."
!Series_summary	"Peripheral blood mononuclear cells collected at ECMO installation were used analyzed."
!Series_overall_design	"Analysis of the cardiogenic shock patients at extracorporeal membrane oxygenation treatment by genome-wide expression and methylation. Transcriptomic profiling and DNA methylation between successful and failure groups were analyzed."
!Series_overall_design	"This submission represents the transcriptome data."
Sample Characteristics Dictionary:
{0: ['course: Acute myocarditis', 'course: Acute myocardial infarction', 'course: Dilated cardiomyopathy, DCMP', 'course:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Dict, Any, Callable
import json

# Set variables based on analysis
is_gene_available = True  # The dataset appears to contain gene expression data based on the Series_overall_design

# 2.1 Data Availability
# Based on the sample characteristics dictionary:
trait_row = 0  # Course of disease (contains Arrhythmia)
age_row = 1    # Age information
gender_row = 2 # Gender information

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 or 1)."""
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if "Arrhythmia" is in the value
    return 1 if "Arrhythmia" in value else 0

def convert_age(value: str) -> float:
    """Convert age value to continuous float."""
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    # Extract the value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Since we don't have the actual clinical data matrix and cannot create one from the
        # sample characteristics dictionary directly, we'll skip this step for now
        print("Cannot extract clinical features: clinical data matrix not available")
        print("The sample characteristics dictionary only provides possible values, not sample-specific data")
        
        # Create a note about this dataset
        note = "Clinical data extraction skipped due to missing proper clinical data matrix format."
        print(note)
    except Exception as e:
        print(f"Error processing clinical data: {e}")
else:
    print("Clinical data not available. Skipping clinical feature extraction.")


Cannot extract clinical features: clinical data matrix not available
The sample characteristics dictionary only provides possible values, not sample-specific data
Clinical data extraction skipped due to missing proper clinical data matrix format.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE93101/GSE93101_series_matrix.txt.gz
Gene data shape: (29363, 33)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers from the previous step

# The identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# Illumina probe IDs are not human gene symbols, they need to be mapped to gene symbols
# These are likely from an Illumina BeadArray microarray platform

# Therefore, gene mapping is required
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC

Column 'Transcript' contains values matching gene data ID pattern


Column 'Species' contains values matching gene data ID pattern


Column 'Source' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Transcript' may contain gene-related information
Sample values: ['ILMN_333737', 'ILMN_333646', 'ILMN_333584']
Column 'ILMN_Gene' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']
Column 'Entrez_Gene_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Symbol' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols
# From the previous output, we can see:
# - 'ID' column contains the probe IDs (ILMN_*)
# - 'Symbol' column contains gene symbols

# 2. Create the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Show the first few rows of the gene expression dataframe
print(f"\nGene expression dataframe shape: {gene_data.shape}")
print("First few rows of gene expression dataframe:")
print(gene_data.head())

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (29377, 2)
First few rows of mapping dataframe:
             ID        Gene
0  ILMN_3166687  ERCC-00162
1  ILMN_3165566  ERCC-00071
2  ILMN_3164811  ERCC-00009
3  ILMN_3165363  ERCC-00053
4  ILMN_3166511  ERCC-00144

Gene expression dataframe shape: (20206, 33)
First few rows of gene expression dataframe:
       GSM2443799  GSM2443800   GSM2443801  GSM2443802   GSM2443803  \
Gene                                                                  
A1BG   129.442547  142.061233   103.958331  137.556161   111.260768   
A1CF   460.835089  324.958428   484.608278  683.954295   657.945539   
A26C3  117.769485   96.247228   143.474170  113.274705   111.123349   
A2BP1  445.728633  419.931068  1118.462328  882.773847   455.880246   
A2LD1  726.498733  129.188312   273.126915  724.925706  1183.148561   

       GSM2443804  GSM2443805  GSM2443806   GSM2443807  GSM2443808  ...  \
Gene                                                                ...   
A1BG   241.7675


Gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE93101.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to liver fibrosis progression, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for liver cirrhosis analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20206, 33)
Gene data shape after normalization: (19445, 33)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE93101.csv
Original clinical data preview:
         !Sample_geo_accession                 GSM2443799  \
0  !Sample_characteristics_ch1  course: Acute myocarditis   
1  !Sample_characteristics_ch1                  age: 33.4   
2  !Sample_characteristics_ch1                  gender: F   
3  !Sample_characteristics_ch1           outcome: Success   

                  GSM2443800                 GSM2443801  \
0  course: Acute myocarditis  course: Acute myocarditis   
1                  age: 51.2                  age: 51.9   
2                  gender: M                  gender: F   
3           outcome: Success           outcome: Failure   

                            GSM2443802                 GSM2443803  \
0  course: Acute myocardial infarction  course: Acute myocarditis   
1                            age: 47.8                  age: 41.5   
2                            gender: M                  gend

Data shape after handling missing values: (33, 19448)
For the feature 'Arrhythmia', the least common label is '1.0' with 2 occurrences. This represents 6.06% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 45.2
  50% (Median): 52.4
  75%: 56.5
Min: 16.1
Max: 78.9
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 10 occurrences. This represents 30.30% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (33, 19448)
Dataset is not usable for analysis. No linked data file saved.
